<a href="https://colab.research.google.com/github/adalves-ufabc/2022.Q2-PLN/blob/main/2022_Q2_PLN_Notebook_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2022.Q2]**
Prof. Alexandre Donizeti Alves

### **Modelagem de Tópicos com LDA**

Neste exemplo faremos a modelagem de tópicos no texto obtido de artigos da Wikipedia. Para baixar a biblioteca `wikipedia`, execute o seguinte comando:

In [1]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=86bacdc6a1fa2be6ad4951ecbd2ba64cdc808c48a19f4f73354243aba7836bec
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


Para visualizar nosso modelo de tópicos, usaremos a biblioteca `pyLDAvis`. Para fazer o download da biblioteca, execute o seguinte comando `pip`:

In [2]:
!pip install pyLDAvis==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.2 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=1ce3729541e9548c814eb82dcc52afa63f1a4fcada7346eb156deaf6ca5cc54d
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


Primeiro importamos as bibliotecas `wikipedia` e `nltk`. Também baixamos as *stop words* em inglês. 


In [3]:
import wikipedia
import nltk

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Em seguida, baixamos o artigo da Wikipedia especificando o tópico para o objeto `page` da biblioteca `wikipedia`. O objeto retornado contém informações sobre a página baixada.

>
Para recuperar o conteúdo da página Web, podemos usar o atributo `content`. O conteúdo de todos os quatro artigos é armazenado na lista denominada `corpus`.

In [4]:
covid = wikipedia.page("covid")
artificial_intelligence = wikipedia.page("Artificial Intelligence")
leonardo_vinci = wikipedia.page("Leonardo da Vinci")
eiffel_tower = wikipedia.page("Eiffel Tower")

corpus = [covid.content, artificial_intelligence.content, leonardo_vinci.content, eiffel_tower.content]

**Pré-processamento dos dados**

Para realizar a modelagem de tópicos via LDA, precisamos de um dicionário de dados e da sacola de palavras (*bag of words*) do corpus. Para isso, precisamos de dados na forma de tokens.

>
Além disso, precisamos remover pontuações e *stop words* de nosso conjunto de dados. Por uma questão de uniformidade, converteremos todos os tokens para minúsculas e também os lematizaremos. Além disso, removeremos todos os tokens com menos de 5 caracteres.

In [6]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # converting to lowercase
        document = document.lower()

        # lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word)  > 5]

        return tokens

In [7]:
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
processed_data = [];
for doc in corpus:
    tokens = preprocess_text(doc)
    processed_data.append(tokens)

No trecho de código acima nós iteramos através da lista de `corpus` que contém os quatro artigos da Wikipedia na forma de strings. Em cada iteração, passamos o documento para o método `preprocess_text` que criamos anteriormente. O método retorna tokens para esse documento específico. Os tokens são armazenados na lista `processed_data`.

>
No final do loop `for`, todos os tokens dos quatro artigos serão armazenados na lista `processed_data`. Agora podemos usar essa lista para criar um dicionário e a sacola de palavras correspondente ao corpus. O seguinte *script* faz isso:

In [9]:
from gensim import corpora

gensim_dictionary = corpora.Dictionary(processed_data)
gensim_corpus = [gensim_dictionary.doc2bow(token, allow_update=True) for token in processed_data]

A seguir, salvaremos nosso dicionário, bem como a sacola de palavras do corpus usando `pickle`. Usaremos o dicionário salvo mais tarde para fazer previsões sobre os novos dados.

In [10]:
import pickle

pickle.dump(gensim_corpus, open('gensim_corpus_corpus.pkl', 'wb'))
gensim_dictionary.save('gensim_dictionary.gensim')

Agora, temos tudo o que é necessário para criar o **modelo LDA** no `Gensim`. Usaremos a classe `LdaModel` do módulo `gensim.models.ldamodel` para criar o modelo LDA. Precisamos passar a sacola de palavras do corpus que criamos anteriormente como o primeiro parâmetro para o construtor `LdaModel`, seguido pelo número de tópicos, o dicionário que criamos anteriormente e o número de passagens (número de iterações para o modelo).

In [11]:
import gensim

lda_model = gensim.models.ldamodel.LdaModel(gensim_corpus, num_topics=4, id2word=gensim_dictionary, passes=20)
lda_model.save('gensim_model.gensim')

Sim, é assim tão simples. No script acima, criamos o modelo LDA de nosso conjunto de dados e o salvamos.

>
A seguir, vamos imprimir 10 palavras para cada tópico. Para fazer isso, podemos usar o método `print_topics`. Execute o seguinte *script*:

In [12]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.049*"leonardo" + 0.014*"painting" + 0.008*"drawing" + 0.004*"florence" + 0.004*"vasari" + 0.004*"artist" + 0.004*"verrocchio" + 0.003*"painter" + 0.003*"including" + 0.003*"century"')
(1, '0.019*"corvus" + 0.018*"corvids" + 0.013*"specie" + 0.012*"magpie" + 0.006*"social" + 0.006*"corvid" + 0.006*"family" + 0.006*"cyanocorax" + 0.005*"corvidae" + 0.004*"ability"')
(2, '0.019*"intelligence" + 0.015*"artificial" + 0.010*"machine" + 0.009*"learning" + 0.009*"problem" + 0.007*"research" + 0.007*"network" + 0.006*"system" + 0.006*"algorithm" + 0.005*"search"')
(3, '0.025*"eiffel" + 0.007*"second" + 0.006*"french" + 0.005*"exposition" + 0.005*"structure" + 0.005*"france" + 0.005*"tallest" + 0.005*"engineer" + 0.004*"design" + 0.004*"construction"')


**IMPORTANTE**: A ordem dos tópicos pode mudar a cada execução do código.
>
O tópico 2 contém palavras como *intelligence*, *artificial*, *machine* etc. Podemos supor que essas palavras pertencem ao tópico relacionado a Inteligência Artificial.

Podemos ver claramente que o modelo LDA identificou com sucesso os quatro tópicos em nosso conjunto de dados.

>
É importante mencionar aqui que o LDA é um algoritmo de aprendizado não supervisionado e, em problemas do mundo real, você não saberá sobre os tópicos do conjunto de dados de antemão. Você simplesmente receberá um corpus, os tópicos serão criados usando LDA e, em seguida, os nomes dos tópicos dependem de você.

Vamos agora criar 8 tópicos usando nosso conjunto de dados. Iremos imprimir 5 palavras por tópico:

In [13]:
lda_model = gensim.models.ldamodel.LdaModel(gensim_corpus, num_topics=8, id2word=gensim_dictionary, passes=15)
lda_model.save('gensim_model.gensim')
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.000*"eiffel" + 0.000*"leonardo" + 0.000*"second" + 0.000*"painting" + 0.000*"french"')
(1, '0.000*"intelligence" + 0.000*"artificial" + 0.000*"machine" + 0.000*"leonardo" + 0.000*"research"')
(2, '0.000*"leonardo" + 0.000*"corvids" + 0.000*"intelligence" + 0.000*"painting" + 0.000*"artificial"')
(3, '0.000*"leonardo" + 0.000*"intelligence" + 0.000*"painting" + 0.000*"drawing" + 0.000*"machine"')
(4, '0.056*"leonardo" + 0.016*"painting" + 0.009*"drawing" + 0.005*"florence" + 0.005*"vasari"')
(5, '0.013*"intelligence" + 0.009*"artificial" + 0.008*"eiffel" + 0.007*"corvus" + 0.006*"machine"')
(6, '0.000*"intelligence" + 0.000*"leonardo" + 0.000*"artificial" + 0.000*"problem" + 0.000*"machine"')
(7, '0.000*"leonardo" + 0.000*"eiffel" + 0.000*"intelligence" + 0.000*"artificial" + 0.000*"corvids"')


Novamente, o número de tópicos que deseja criar depende de você. Continue tentando números diferentes até encontrar tópicos adequados. Para nosso conjunto de dados, o número adequado de tópicos é 4, pois já sabemos que nosso corpus contém palavras de quatro artigos diferentes. Reverta para quatro tópicos executando o seguinte *script*:

Desta vez, você verá resultados diferentes, uma vez que os valores iniciais para os parâmetros LDA são escolhidos aleatoriamente. Os resultados desta vez são os seguintes:

In [22]:
lda_model = gensim.models.ldamodel.LdaModel(gensim_corpus, num_topics=4, id2word=gensim_dictionary, passes=20)
lda_model.save('gensim_model.gensim')
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.019*"intelligence" + 0.015*"artificial" + 0.010*"machine" + 0.009*"learning" + 0.009*"problem" + 0.007*"research" + 0.007*"network" + 0.006*"system" + 0.006*"algorithm" + 0.005*"search"')
(1, '0.049*"leonardo" + 0.014*"painting" + 0.008*"drawing" + 0.004*"florence" + 0.004*"vasari" + 0.004*"artist" + 0.004*"verrocchio" + 0.003*"painter" + 0.003*"including" + 0.003*"century"')
(2, '0.025*"eiffel" + 0.007*"second" + 0.006*"french" + 0.005*"structure" + 0.005*"exposition" + 0.005*"france" + 0.005*"tallest" + 0.005*"engineer" + 0.004*"design" + 0.004*"construction"')
(3, '0.019*"corvus" + 0.018*"corvids" + 0.013*"specie" + 0.012*"magpie" + 0.006*"social" + 0.006*"corvid" + 0.006*"family" + 0.006*"cyanocorax" + 0.005*"corvidae" + 0.004*"ability"')


**Avaliando o modelo LDA**

Conforme mencionado anteriormente, os modelos de aprendizagem não supervisionados são difíceis de avaliar, uma vez que não existe uma verdade concreta contra a qual possamos testar a saída de nosso modelo.

>
Suponha que temos um novo documento de texto e queremos encontrar seu tópico usando o modelo LDA que acabamos de criar, podemos fazer isso usando o seguinte *script*:

In [23]:
test_doc = 'Great structures are build to remember an event happened in the history.'
test_doc = preprocess_text(test_doc)
bow_test_doc = gensim_dictionary.doc2bow(test_doc)

print(lda_model.get_document_topics(bow_test_doc))

[(0, 0.06355073), (1, 0.0635583), (2, 0.51557046), (3, 0.35732052)]


No *script* acima, criamos uma string, criamos sua representação no dicionário e então convertemos a string no corpus do saco de palavras. A representação do saco de palavras é então passada para o método `get_document_topics`. 

A saída mostra que há 51,55% de chance de que o novo documento pertença ao  tópico 2 (consulte as palavras para o tópico 2 na última saída). Da mesma forma, há 35,73% de chance de este documento pertencer ao tópico 3 (covid). Se olharmos para o tópico 2, ele contém palavras relacionadas à Torre Eiffel. Nosso documento de teste também contém palavras relacionadas a estruturas e edifícios. Portanto, foi atribuído o tópico 2.

Outra forma de avaliar o modelo LDA é por meio de `Perplexity` (Perplexidade) e `Coherence Score` (Coerência).

Como regra geral para um bom modelo de LDA, a pontuação de perplexidade deve ser baixa, enquanto a coerência deve ser alta. A biblioteca Gensim possui uma classe `CoherenceModel` que pode ser usada para encontrar a coerência do modelo LDA. Para perplexidade, o objeto `LdaModel` contém o método `log_perplexity` que pega uma sacola de palavras como parâmetro e retorna a perplexidade correspondente.

In [24]:
print('\nPerplexity:', lda_model.log_perplexity(gensim_corpus))

from gensim.models import CoherenceModel

coherence_score_lda = CoherenceModel(model=lda_model, texts=processed_data, dictionary=gensim_dictionary, coherence='c_v')
coherence_score = coherence_score_lda.get_coherence()

print('\nCoherence Score:', coherence_score)


Perplexity: -7.736988352824177

Coherence Score: 0.6435901836032951


**Visualizando o modelo LDA**

In [25]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

Para visualizar nossos dados, podemos usar a biblioteca `pyLDAvis` que baixamos no início.

In [26]:
gensim_dictionary = gensim.corpora.Dictionary.load('gensim_dictionary.gensim')
gensim_corpus = pickle.load(open('gensim_corpus_corpus.pkl', 'rb'))
lda_model = gensim.models.ldamodel.LdaModel.load('gensim_model.gensim')

In [27]:
import pyLDAvis.gensim

lda_visualization = pyLDAvis.gensim.prepare(lda_model, gensim_corpus, gensim_dictionary, sort_topics=False)
pyLDAvis.display(lda_visualization)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


Cada círculo na imagem acima corresponde a um tópico.

>
A distância entre os círculos mostra como os tópicos são diferentes uns dos outros. Os círculos também podem estar sobrepostos, indicando que os tópicos têm muitas palavras em comum.

Se você passar o mouse sobre qualquer palavra à direita, verá apenas o círculo do tópico que contém a palavra. Uma palavra pode estar relacionada a mais de um tópico, assim mais de um círculo aparecerá.

Da mesma forma, se você clicar em qualquer um dos círculos, uma lista dos termos mais frequentes para aquele tópico aparecerá à direita junto com a frequência de ocorrência naquele mesmo tópico.

**Mais informações:**

> https://stackabuse.com/python-for-nlp-working-with-the-gensim-library-part-2/